# 🧪 Lab 10: Memory + Multi-Tool Orchestration (LangGraph)

**Goal:** Build a travel assistant that remembers preferences across turns and uses multiple tools.


## Setup
```bash
pip install -U langgraph langchain langchain-openai typing_extensions
export OPENAI_API_KEY="your_key"
```


In [ ]:
from typing import Dict, Tuple
import re
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import AnyMessage, HumanMessage
from typing_extensions import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver


## Define Tools

In [ ]:
@tool("get_weather", return_direct=False)
def get_weather(city: str) -> str:
    weather_db: Dict[str, str] = {
        "paris": "sunny, 24°C",
        "chicago": "cloudy, 18°C",
        "mumbai": "rainy, 30°C",
        "london": "overcast, 17°C",
        "tokyo": "clear, 26°C",
    }
    c = city.strip().lower()
    if c in weather_db:
        return f"The weather in {city} is {weather_db[c]}."
    return f"No weather found for '{city}'. Assume mild (22°C) and clear for demo."


In [ ]:
@tool("mini_wiki", return_direct=False)
def mini_wiki(topic: str) -> str:
    kb: Dict[str, str] = {
        "paris": "Paris is the capital of France, known for the Eiffel Tower and the Louvre.",
        "london": "London is the capital of the UK, home to the British Museum and the Thames.",
        "tokyo": "Tokyo blends tradition and technology; famous for Shibuya Crossing and Ueno Park.",
        "mumbai": "Mumbai is India's financial hub, known for Marine Drive and film industry.",
        "chicago": "Chicago sits on Lake Michigan; known for the Riverwalk and deep-dish pizza.",
    }
    return kb.get(topic.strip().lower(), "No entry found in mini_wiki. Try 'Paris' or 'Chicago'.")


In [ ]:
def _parse_city_weather(query: str):
    q = query.strip()
    if ";" in q or "city=" in q.lower():
        parts = [p.strip() for p in q.split(";")]
        city, weather = "", ""
        for p in parts:
            if "=" in p:
                k, v = p.split("=", 1)
                k = k.strip().lower(); v = v.strip()
                if k == "city":
                    city = v
                elif k == "weather":
                    weather = v
        if not city:
            city = re.split(r"[;,]", q)[0].replace("city", "").replace("=", "").strip()
        return city, weather
    return q, ""


In [ ]:
@tool("suggest_city_activities", return_direct=False)
def suggest_city_activities(query: str) -> str:
    catalog = {
        "chicago": {
            "indoor": ["Art Institute of Chicago", "Museum of Science and Industry", "Field Museum"],
            "outdoor": ["Chicago Riverwalk", "Millennium Park", "Navy Pier"],
        },
        "paris": {
            "indoor": ["Louvre Museum", "Musée d'Orsay"],
            "outdoor": ["Seine River Walk", "Jardin du Luxembourg"],
        },
        "london": {
            "indoor": ["British Museum", "Tate Modern"],
            "outdoor": ["Hyde Park", "South Bank Walk"],
        },
        "tokyo": {
            "indoor": ["teamLab Planets", "Tokyo National Museum"],
            "outdoor": ["Ueno Park", "Shibuya Crossing Walk"],
        },
        "mumbai": {
            "indoor": ["Chhatrapati Shivaji Maharaj Vastu Sangrahalaya", "Phoenix Mall"],
            "outdoor": ["Marine Drive", "Sanjay Gandhi National Park"],
        },
    }
    city, weather = _parse_city_weather(query)
    c = city.strip().lower()
    if not c:
        return "Please provide a city name (e.g., 'city=Paris; weather=sunny, 24°C')."
    data = catalog.get(c)
    if not data:
        return "General: Indoor - local museum or aquarium. Outdoor - central park or riverfront walk."
    w = weather.lower()
    indoor_first = any(k in w for k in ["rain", "storm"]) or ("overcast" in w and "cold" in w)
    if indoor_first:
        indoor = data["indoor"][0]; outdoor = data["outdoor"][0]
    elif any(k in w for k in ["sunny", "clear"]):
        outdoor = data["outdoor"][0]; indoor = data["indoor"][0]
    else:
        indoor = data["indoor"][0]; outdoor = data["outdoor"][0]
    return f"City: {city}. Indoor: {indoor}. Outdoor: {outdoor}. (Weather-aware heuristics.)"


In [ ]:
@tool("calculator", return_direct=False)
def calculator(expression: str) -> str:
    import math, re
    if not re.fullmatch(r"[0-9+\-*/(). %\s]+", expression):
        return "Calculator error: invalid characters."
    try:
        result = eval(expression, {"__builtins__": {}}, {"math": math})
        return str(result)
    except Exception as e:
        return f"Calculator error: {e}"


## Build Graph

In [ ]:
TOOLS = [get_weather, mini_wiki, suggest_city_activities, calculator]

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

def agent_node(state: AgentState) -> AgentState:
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(TOOLS))

def route_from_agent(state: AgentState):
    last = state["messages"][-1]
    tc = getattr(last, "tool_calls", None) or getattr(getattr(last, "additional_kwargs", {}), "get", lambda *_: None)("tool_calls")
    return "tools" if tc else END

graph.set_entry_point("agent")
graph.add_conditional_edges("agent", route_from_agent)
graph.add_edge("tools", "agent")

checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)


## Demo

In [ ]:
cfg = {"configurable": {"thread_id": "t-notebook-lab10"}}
msgs = [
    HumanMessage(content="Hi, I'm Priya. I prefer indoor activities and budget-friendly options."),
    HumanMessage(content="Plan a 1-day trip in Paris considering the weather; include one indoor and one outdoor pick."),
    HumanMessage(content="What did I say my preference was?"),
    HumanMessage(content="Now do the same for Chicago."),
    HumanMessage(content="Compute 25 + 18 + 12.5 and tell me only the number."),
]
for m in msgs:
    out = app.invoke({"messages": [m]}, cfg)
    display({"user": m.content, "assistant": out["messages"][-1].content})
